In [1]:
import cv2
import os
import numpy as np

In [11]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

input_dir = os.path.join(BASE_DIR, "data", "ct_scan")
output_dir = os.path.join(BASE_DIR, "data", "preprocessed")

# Create output folders
os.makedirs(os.path.join(output_dir, "Tumor"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "NORMAL"), exist_ok=True)

In [7]:
def add_contrast(img):
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  img_clahe = clahe.apply(img)
  return img_clahe

In [8]:
def crop_brain_ct(img):
    norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    _, thresh = cv2.threshold(norm, 0, 255, cv2.THRESH_OTSU)
    conts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(conts) == 0:
        return img

    c = max(conts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    return img[y:y+h, x:x+w]


In [12]:
print("INPUT DIR:", input_dir)
print("Exists:", os.path.exists(input_dir))

print("Tumor exists:", os.path.exists(os.path.join(input_dir, "Tumor")))


for cls in ["Tumor", "NORMAL"]:
    src = os.path.join(input_dir, cls)
    dst = os.path.join(output_dir, cls)

    for filename in os.listdir(src):
        path = os.path.join(src, filename)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print("Skipping:", filename)
            continue

        # 1. Apply CT windowing
        img = add_contrast(img)

        # 2. Crop to skull/brain
        img = crop_brain_ct(img)

        # 3. Resize to model size
        img = cv2.resize(img, (224, 224))

        # 4. Expand to 3-channel
        img = np.stack([img, img, img], axis=-1)  # shape: (224,224,3)

        # 5. Save final output
        save_path = os.path.join(dst, filename)
        cv2.imwrite(save_path, img)

        print("Processed:", filename)

print("✔ Finished preprocessing CT scans.")

INPUT DIR: c:\Users\User\Documents\DS PROJECT 2603\DSGp-BTD-git\DSGP-Brain-Tumor-Detection\data\ct_scan
Exists: True
Tumor exists: True
Processed: ct_tumor (1).jpg
Processed: ct_tumor (1).png
Processed: ct_tumor (10).jpg
Processed: ct_tumor (10).png
Processed: ct_tumor (100).jpg
Processed: ct_tumor (100).png
Processed: ct_tumor (1000).jpg
Processed: ct_tumor (1001).jpg
Processed: ct_tumor (1002).jpg
Processed: ct_tumor (1003).jpg
Processed: ct_tumor (1004).jpg
Processed: ct_tumor (1005).jpg
Processed: ct_tumor (1006).jpg
Processed: ct_tumor (1007).jpg
Processed: ct_tumor (1008).jpg
Processed: ct_tumor (1009).jpg
Processed: ct_tumor (101).jpg
Processed: ct_tumor (101).png
Processed: ct_tumor (1010).jpg
Processed: ct_tumor (1011).jpg
Processed: ct_tumor (1012).jpg
Processed: ct_tumor (1013).jpg
Processed: ct_tumor (1014).jpg
Processed: ct_tumor (1015).jpg
Processed: ct_tumor (1016).jpg
Processed: ct_tumor (1017).jpg
Processed: ct_tumor (1018).jpg
Processed: ct_tumor (1019).jpg
Processed: